In this notebook, we are interested in comparing the fluctation in the rate of occurrence of the different clusters with meteorological data, through a regression exercise. The regression will be conducted through three regressorse easily integrated through the ``scikit-learn`` environment, mainly ``RandomForestRegressor``, ``GradientBoostingRegressor`` and ``XGBRegressor``.

I have pre-cleaned the meteorological data and gathered it in ``output/meteorological_data.csv``, while the rate of occurrence is stored in ``output/countdf.csv``. You can see how it was computed in the previous notebook, ``2. Clustering.ipynb``.

In [36]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import shap

np.random.seed(seed=42069)


In [37]:
countdf = pd.read_csv('output/countdf.csv', index_col=0)
countdf.index = pd.to_datetime(countdf.index)

meteo = pd.read_csv('output/meteorological_data.csv', index_col=0)
meteo.index = pd.to_datetime(meteo.index)

I have highlighted three consecutive sequences with valid data for our seismic and meteorological data. The workflow for this work is divided in the following steps:

 1. divide the time series in consecutive, valid sequences,
 2. transform the target variable through a log<sub>10</sub> transform and a smoothing window,
 3. compute statistical moments of the meteorological variables through a smoothing window, 
 4. split the data in training and testing sets with respect to the temporal nature of the problem,
 5. perform a regression with the different regressors such that ``cluster occurence = f(meteorological variables)``, and
 6. look at the importance of the variables through shapley values.

In [38]:
# those are the three, long, sequences with valid data
sequences = ('2020-01-01 0:00:00', '2020-10-06 20:00:00'), ('2021-01-29 01:15:00', '2021-05-26 11:30:00'), ('2021-06-23 20:00:00', '2021-09-14 06:30:00')

# smoothing windows of one hour, one day and one week
windows = [4, 96, 672]  
window_labels = ['1h', '1 day', '1 wk.']

# we define a smoothing window for the count of one day, this can be easily changed
smooth_window = 48

dt = countdf.index[1] - countdf.index[0]
x_sequences = []
y_sequences_smooth = []
y_sequences_raw = []

# we slice the data in our three consecutive sequences and transform the rate of occurence as we go
for t1, t2 in sequences:
    
    # we slice the data we're interested in
    y = countdf[(t1 < countdf.index) & (countdf.index < t2)]
    
    # we apply a few transforms, keep the true and smoothed data
    ytrue = y.copy()
    y = y.rolling(window=smooth_window).mean()
    y = y.dropna(axis=0)
    y = np.log(y + 1)

    # we use pandas' indexing, making sure that we go back far enough for computing the longest smoothing window
    valid_times = pd.date_range(y.index[0] - max(windows) * dt, y.index[-1], freq='15min')
    x = meteo.loc[valid_times]
    x_sequences.append(x)
    y_sequences_smooth.append(y)
    y_sequences_raw.append(ytrue)

In [39]:
# here we computing the moving statistical moments for the meteorological data
nseq = len(x_sequences)

dfs = []

if window_labels is None:
    window_labels = windows

for i in range(nseq):
    x, y = x_sequences[i], y_sequences_smooth[i]
    df = pd.DataFrame(index=y.index)

    for j, w in enumerate(windows):
        roll = x.rolling(window=w)

        maximum = roll.max()
        minimum = roll.min()
        avg = roll.mean()
        std = roll.std()  

        avg.columns = avg.columns + f', mean {window_labels[j]}'
        std.columns = std.columns + f', std. {window_labels[j]}'

        df = df.join((avg, std,), how='inner')

    df = df.dropna(axis=1, how='any')
    dfs.append(df)

The following cell does a few things, defining helper functions to split the data in trainining and testing sets through ``get_seasonal_train_test_indices``, as well as fitting a given regressor according to those indices through ``fit_clf_seasonally``. The function ``df_dict_to_excel`` saves a dictionary of dataframes in an excel file, where every sheet is a dataframe. We also define the dictionaries containing the parameters of every regressor acquired through a gridsearch.

In [47]:
def get_seasonal_train_test_indices(X, nsplits=5, train_len=96 * 7 * 4, test_len=96 * 7):
    train_ii = []
    test_ii = []

    timesplit = np.floor(len(X) / (train_len + test_len)).astype(int)

    t0 = 0
    for i in range(timesplit):
        t1 = t0 + train_len
        t2 = t1 + test_len

        if i == timesplit - 1:
            train_ii.append(np.hstack((np.arange(t0, t1), np.arange(t2, len(X)))))
        else:
            train_ii.append(np.arange(t0, t1))

        test_ii.append(np.arange(t1, t2))
        t0 = t2

    if nsplits == 1:
        return np.hstack(train_ii), np.hstack(test_ii)
    else:

        from sklearn.utils import shuffle
        train_ii = shuffle(np.hstack(train_ii))
        test_ii = shuffle(np.hstack(test_ii))

        return zip(np.array_split(train_ii, nsplits), np.array_split(test_ii, nsplits))
    
def fit_clf_seasonally(clf, X, y, nsplits=5, train_len=96 * 7 * 4, test_len=96 * 7):
    kfold = get_seasonal_train_test_indices(X, nsplits=nsplits, train_len=train_len, test_len=test_len)
    indices = [i for i in kfold]

    scores = np.zeros((nsplits, 2))

    from sklearn.base import clone

    classifiers = np.zeros(nsplits, dtype='object')
    for i in range(nsplits):
        classifiers[i] = clone(clf)

    for i, (train_index, test_index) in enumerate(indices):

        clf = classifiers[i]

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf.fit(X_train, y_train)

        train_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)

        scores[i, 0] = train_score
        scores[i, 1] = test_score

        explainer = shap.TreeExplainer(clf)
        clf.shap_values = explainer.shap_values(X_train, y_train)

        clf.X_train = X[train_index]

    train_index = np.hstack([i[0] for i in indices])
    test_index = np.hstack([i[1] for i in indices])

    return classifiers, scores, train_index, test_index

def df_dict_to_excel(df_dict, path, header=True, index=True):

    writer = pd.ExcelWriter(path, engine='xlsxwriter')

    for tab_name, df in df_dict.items():
        df.to_excel(writer, sheet_name=tab_name, header=header, index=index)
    
    wb = writer.book
    format = wb.add_format({'num_format': '0.00'})
    
    for ws in wb.worksheets():
        df = df_dict[ws.name]
        ws.set_column(1, len(df.columns) + 1, cell_format=format)
        length_list = [max([len(i) for i in df.index])] + [len(x) for x in df.columns]
        for i, width in enumerate(length_list):
            ws.set_column(i, i, width)
            ws.conditional_format(1, i, len(df) + 1, i, {'type': '3_color_scale'})

    #writer._save()
    writer.close()
    
rf = {'max_depth': 30,
        'min_samples_leaf': 0.05,
        'min_samples_split': 0.05,
        'n_estimators': 250,
        'max_features': 'sqrt'}

gb = {'learning_rate': 0.05,
        'max_depth': 10,
        'min_samples_leaf': 0.05,
        'min_samples_split': 0.005,
        'n_estimators': 250,
        'max_features': 'sqrt',
        'subsample': 0.6}

xgb = {'gamma': 0.5,
        'max_depth': 15,
        'min_child_weight': 5,
        'n_estimators': 250,
        'subsample': 0.3,
        'learning_rate': 0.05}
    
models = [GradientBoostingRegressor(**gb), XGBRegressor(**xgb), RandomForestRegressor(**rf)]
                                    
name_dict = {'RandomForestRegressor': 'Random forests',
             'GradientBoostingRegressor': 'Gradient boosting',
             'XGBRegressor': 'XGBoost'}

In [48]:
for df in y_sequences_raw:
    print(len(df))

7776
11272
7913


In [49]:
def print_scores(scores):
    print(f'average: {scores.mean():.2f}, maximum: {scores.max():.2f}, '
          f'minimum: {scores.min():.2f}, std: {scores.std():.2f}')
    
nclusters = countdf.shape[1]

train_scores = np.zeros((nclusters, len(models)))
test_scores = np.zeros((nclusters, len(models)))

df_dict = {}

preds = np.zeros((len(models), nclusters, len(dfs)), dtype='object')

X = pd.concat(dfs)
y = pd.concat(y_sequences_smooth)

for c in range(nclusters):
    print(f'Training cluster {c}')
    suby = y[f'CLUSTER_{c}']

    dataframes = []
    for i, clf in enumerate(models):
        tag = clf.__class__.__name__
        name = name_dict[tag]
        title = f'{name}, cluster {c}'
        print(f'Training {name}')

        clfs, scores, train_index, test_index = fit_clf_seasonally(clf, X.values, suby.values,
                                                                    train_len=96 * 7 * 2, test_len=96 * 3)
        print('TRAIN')
        print_scores(scores[:, 0])
        print('TEST')
        print_scores(scores[:, 1])
        print()

        train_scores[c, i] = scores[:, 0].mean()
        test_scores[c, i] = scores[:, 1].mean()

        coefs = np.vstack([clf.shap_values for clf in clfs])

        means = np.stack([clf.shap_values.mean(axis=0) for clf in clfs])
        order = np.argsort(np.abs(means.mean(axis=0)))[::-1]
        s = pd.Series(np.abs(means.mean(axis=0))[order], index=X.columns[order], name=name)
        s.name = name
        dataframes.append(s.to_frame())
        print()

    df_dict[f'Cluster {c}'] = pd.concat(dataframes, axis=1)

test_scores[test_scores < 0] = 0
weights = test_scores/(test_scores.sum(axis=1))[:, None]  # (1/test_scores) / (1/test_scores).sum(axis=1)[:, None]

for i, (key, df) in enumerate(df_dict.items()):
    x = np.abs(df.iloc[:, :len(models)])
    x = (x - x.min()) / (x.max() - x.min())
    w = weights[i][None, :]
    wmean = np.sum(w * x, axis=1).values[:, None]
    df['Valeur absolue moyenne'] = x.mean(axis=1)
    df['Écart type valeur absolue'] = x.std(axis=1)
    df['Pondération valeur absolue moyenne'] = wmean
    df['Pondération écart type valeur absolue'] = np.sqrt((w * (x - wmean) ** 2).sum(axis=1))
    df['Maximum valeur absolue'] = x.max(axis=1)
    df['Minimum valeur absolue'] = x.min(axis=1)
    df.iloc[:, :len(models)] = x
    df_dict[key] = df.sort_values('Pondération valeur absolue moyenne', ascending=False)

test_score_df = pd.DataFrame(test_scores,
                                index=[f'Cluster {i}' for i in range(nclusters)],
                                columns=list(df_dict.values())[0].columns[:len(models)])
train_score_df = pd.DataFrame(train_scores,
                                index=[f'Cluster {i}' for i in range(nclusters)],
                                columns=list(df_dict.values())[0].columns[:len(models)])
df_dict['Training score'] = train_score_df
df_dict['Testing score'] = test_score_df

df_dict_to_excel(df_dict, path='outputs/coefficients_ranking_3clusters.xlsx')

Training cluster 0
Training Gradient boosting
TRAIN
average: 0.74, maximum: 0.75, minimum: 0.74, std: 0.01
TEST
average: 0.19, maximum: 0.25, minimum: 0.10, std: 0.06


Training XGBoost


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


TRAIN
average: 0.74, maximum: 0.75, minimum: 0.72, std: 0.01
TEST
average: 0.29, maximum: 0.40, minimum: 0.25, std: 0.05


Training Random forests
TRAIN
average: 0.33, maximum: 0.34, minimum: 0.32, std: 0.01
TEST
average: 0.16, maximum: 0.18, minimum: 0.14, std: 0.02


Training cluster 1
Training Gradient boosting
TRAIN
average: 0.86, maximum: 0.86, minimum: 0.86, std: 0.00
TEST
average: 0.49, maximum: 0.56, minimum: 0.45, std: 0.04


Training XGBoost


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


TRAIN
average: 0.89, maximum: 0.89, minimum: 0.88, std: 0.00
TEST
average: 0.47, maximum: 0.52, minimum: 0.40, std: 0.04


Training Random forests
TRAIN
average: 0.61, maximum: 0.62, minimum: 0.59, std: 0.01
TEST
average: 0.39, maximum: 0.44, minimum: 0.34, std: 0.04


Training cluster 2
Training Gradient boosting
TRAIN
average: 0.74, maximum: 0.75, minimum: 0.73, std: 0.01
TEST
average: 0.37, maximum: 0.41, minimum: 0.33, std: 0.03


Training XGBoost


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


TRAIN
average: 0.84, maximum: 0.85, minimum: 0.84, std: 0.00
TEST
average: 0.37, maximum: 0.40, minimum: 0.34, std: 0.02


Training Random forests
TRAIN
average: 0.40, maximum: 0.41, minimum: 0.39, std: 0.00
TEST
average: 0.26, maximum: 0.28, minimum: 0.22, std: 0.02




In [53]:
path = r'output\coefficients_ranking_3clusters.xlsx'

df_dict = pd.read_excel(path, sheet_name=None)

nmax = 10
y = np.arange(nmax) + 1

fig, axs = plt.subplots(1, 3, figsize=(10, 3))

for c in range(3):
    dfi = df_dict[f'Cluster {c}']
    
    variables = dfi.iloc[:nmax, 0]
    mean = dfi['Pondération valeur absolue moyenne'].iloc[:nmax]
    std = dfi['Pondération écart type valeur absolue'].iloc[:nmax]
    
    axs[c].barh(y, mean, fc=colors[c], alpha=0.5, ec=colors[c], lw=1, xerr=np.vstack((np.zeros(nmax), std)), ecolor=colors[c], error_kw=dict(alpha=0.5))
    
    for i in range(nmax):
        axs[c].text(0.01, y[i], variables[i], va='center')
    axs[c].invert_yaxis()
    axs[c].set_yticks(y)
    axs[c].set_xlim(0, 1)

    axs[c].xaxis.tick_top()
    axs[c].set_xticks([0, 0.5, 1])
    axs[c].spines[['right', 'bottom']].set_visible(False)
annotate_axs(axs, x=0.99, y=0.99)
#fig.savefig(savepath + 'variables.pdf', bbox_inches='tight')



FileNotFoundError: [Errno 2] No such file or directory: 'output\\coefficients_ranking_3clusters.xlsx'